# 2022-07-16 -- `unlabeled yale fossils dataset` -- playground notebook

Created on: Saturday July 16th, 2022  
Created by: Jacob A Rose

- using `torchshow`

In [ ]:
#!pip3 install torchshow

In [ ]:
from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)


In [ ]:
dir(pd.options.display)

In [ ]:
%load_ext autoreload
%autoreload 2


from omegaconf import DictConfig, OmegaConf
import os
from rich import print as pp

import numpy as np
from typing import *
import inspect
from tqdm.auto import tqdm
from pathlib import Path
import logging
from imutils.catalog_registry import available_datasets

In [ ]:
from imutils.big.common_catalog_utils import DataETL


LOAD_LEAVESDB = False
LOAD_YALE_FULL = True

In [ ]:
# DataETL.import_dataset_state(

In [ ]:
if LOAD_LEAVESDB:
    dataset_catalog_dir = "/media/data_cifs/projects/prj_fossils/users/jacob/data/leavesdb-v1_1"
    dataset_names = sorted(os.listdir(dataset_catalog_dir))
    main_datasets = [d for d in dataset_names if (not "_minus_" in d) and (not "_w_" in d) and (not "original" in d) and ("512" in d) and ("family" in d)]

    %%time
    # data_dirs = [Path(dataset_catalog_dir, d) for d in main_datasets]
    data_assets = [
        {"config_path": Path(dataset_catalog_dir, d, "CSVDataset-config.yaml"),
         "dataset_name": d}
        for d in main_datasets
    ]

    datasets = {}
    for asset in tqdm(data_assets):
        datasets[asset["dataset_name"]] = DataETL.import_dataset_state(**asset)
        pp(asset["dataset_name"])

    print(len(datasets))
    # datasets = {k:v for k, v in datasets.items() if "512" in k}
    pp(list(datasets.keys()))

## yale fossils

In [ ]:
from PIL.ImageStat import Stat
import meerkat as mk
# LOAD_YALE_FULL

In [ ]:
# yale_fossil_dir = "/media/data_cifs/projects/prj_fossils/data/yale_fossil/outblur_yale"

yale_fossil_dirs = {"outblur_yale": "/media/data_cifs/projects/prj_fossils/data/yale_fossil/outblur_yale",
                    "outblur_yale_tt5_lt3": "/media/data_cifs/projects/prj_fossils/data/yale_fossil/outblur_yale_tt5_lt3",
                    "yale":"/media/data_cifs/projects/prj_fossils/data/yale_fossil/yale",
                    "yale_full":"/media/data_cifs/projects/prj_fossils/data/yale_full"
                   }

dir_keys = [
    "outblur_yale",
    "outblur_yale_tt5_lt3",
    "yale",
    "yale_full"
]


In [ ]:
# dir(mk.config.DisplayOptions)
display_res = 512
# print(mk.config.DisplayOptions.max_image_width)
mk.config.DisplayOptions.max_image_width = display_res
mk.config.DisplayOptions.max_image_height = display_res
print(f"{mk.config.DisplayOptions.max_image_width=}")

In [ ]:
%%time

from typing import *
# fpaths = {}
# dps = {}

    
def extract_dataset_from_file_list(fpaths: List) -> mk.DataPanel:
    dp = mk.DataPanel({
        "paths" : fpaths,
        "file_ids" : [Path(p).stem for p in fpaths],
        "imgs" : mk.ImageColumn.from_filepaths(fpaths)
    })
    dp["height"] = dp["imgs"].to_lambda(fn=lambda x: x.height)
    # dp[k]["mean"] = dp[k]["imgs"].to_lambda(fn=lambda x: Stat(x).mean)
    dp["width"] = dp["imgs"].to_lambda(fn=lambda x: x.width)
    dp["aspect_ratio"] = dp[["height", "width"]].to_lambda(fn=lambda x: x["height"] / x["width"])
    
    return dp

    
def extract_dataset_from_directory(parent_dir) -> mk.DataPanel:
    
    blacklist = [".ipynb_checkpoints"]
    fpaths = sorted([os.path.join(parent_dir, p) for p in os.listdir(parent_dir) if p not in blacklist])
    
    return extract_dataset_from_file_list(fpaths)

In [ ]:
yale_full = extract_dataset_from_directory(parent_dir=yale_fossil_dirs["yale_full"])
dp = yale_full
dp.lz[:3]

In [ ]:
# dir(dp["height"])

# %%time

# # dp["height"] = mk.PandasSeriesColumn(dp["height"].copy())
# height = dp.lz[:10]["height"].copy()

# height

# %%time

# # dp["height"] = mk.PandasSeriesColumn(dp["height"].copy())
# height = dp["height"].lz[:10].copy()
# height

# %%time

# # dp["height"] = mk.PandasSeriesColumn(dp["height"].copy())
# height = dp["height"].copy().lz[:10]
# height

# height._get_state()

# height._get

# %%time

# # dp["height"] = mk.PandasSeriesColumn(dp["height"].copy())
# height = dp["height"].copy()

# height

# %%time
# height = dp["height"].view()
# height
# dp["height"] = dp["height"].copy()
# dp["height"] = mk.PandasSeriesColumn(dp["height"].copy())
# dp.head()

# height = dp["height"].data
# height

### Export parsed file ids catalog to tsv

In [ ]:
dp[["paths", "file_ids"]].to_pandas().to_csv("/media/data_cifs/projects/prj_fossils/data/raw_data/2022-yale_fossil/metadata-merged/0_image_file_ids.tsv", sep="\t", na_rep="NaN")

### extracting yale metadata

In [ ]:
# metadata_dir = "/media/data_cifs/projects/prj_fossils/data/raw_data/2022-yale_fossil/metadata/"
metadata_dir = "/media/data_cifs/projects/prj_fossils/data/raw_data/2022-yale_fossil/metadata-clean"
occurrence_df = pd.read_csv(Path(metadata_dir, "occurrence-clean.txt"), delimiter="\t")#, nrows=9449)
# occurrence_df = pd.read_json(Path(metadata_dir, "occurrence.txt"), skiprows)
occurrence_df.head()

In [ ]:
multimedia_df = pd.read_csv(Path(metadata_dir, "multimedia-clean.txt"), delimiter="\t", nrows=9449)
# occurrence_df = pd.read_json(Path(metadata_dir, "occurrence.txt"), skiprows)

multimedia_df.head()

In [ ]:
multimedia_df.columns

occurrence_df.columns

In [ ]:
from matplotlib_venn import venn2
# venn2(subsets = (3, 2, 1))
subsets = (set(multimedia_df.columns.values.tolist()),
           set(occurrence_df.columns.values.tolist()))
venn2(subsets = subsets)

In [ ]:
subsets[0].intersection(subsets[1])

In [ ]:
from supervenn import supervenn
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%%time

merged = multimedia_df.merge(occurrence_df, how="inner", on="id")
print(f"{multimedia_df.shape=}, {occurrence_df.shape=}, {merged.shape=}")

merged = merged[sorted(merged.columns)]

merged.describe(include='all')

In [ ]:
merged_cols = ['id',
               'identifier',
               'catalogNumber',
               'year',
               'datasetID',
               'datasetName',
               'dateIdentified',
               'class',
               'kingdom',
               'order',
               'family',
               'genus',
               'subgenus',
               'specificEpithet',
               'phylum',
               'scientificName',
               'taxonRank',
               'institutionID',
               'locality',
               'accessURI',
               'associatedMedia',
               'associatedOccurrences',
               'associatedReferences',
               'associatedSequences',
               'associatedTaxa',
               'MetadataDate',
               'WebStatement',
 'accessRights',
 'basisOfRecord',
 'behavior',
 'bibliographicCitation',
 'collectionCode',
 'collectionID',
 'comments',
 'continent',
 'coordinateUncertaintyInMeters',
 'country',
 'county',
 'creator',
 'creator.1',
 'dataGeneralizations',
 'day',
 'decimalLatitude',
 'decimalLongitude',
 'digitizationDate',
 'disposition',
 'dynamicProperties',
 'earliestAgeOrLowestStage',
 'earliestEpochOrLowestSeries',
 'earliestPeriodOrLowestSystem',
 'eventDate',
 'eventTime',
 'fieldNumber',
 'format',
 'format.1',
 'formation',
 'geodeticDatum',
 'georeferenceProtocol',
 'georeferenceRemarks',
 'georeferenceSources',
 'georeferencedBy',
 'georeferencedDate',
 'group',
 'habitat',
 'higherClassification',
 'higherGeography',
 'identificationQualifier',
 'identificationReferences',
 'identificationRemarks',
 'identifiedBy',
 'individualCount',
 'informationWithheld',
 'infraspecificEpithet',
 'institutionCode',
 'language',
 'license',
 'lifeStage',
 'lowestBiostratigraphicZone',
 'maximumDepthInMeters',
 'maximumElevationInMeters',
 'member',
 'metadataLanguage',
 'minimumDepthInMeters',
 'minimumElevationInMeters',
 'modified',
 'month',
 'municipality',
 'nomenclaturalCode',
 'occurrenceID',
 'occurrenceRemarks',
 'otherCatalogNumbers',
 'ownerInstitutionCode',
 'preparations',
 'previousIdentifications',
 'providerManagedID',
 'recordNumber',
 'recordedBy',
 'references',
 'reproductiveCondition',
 'rights',
 'rights.1',
 'rightsHolder',
 'scientificNameAuthorship',
 'sex',
 'stateProvince',
 'taxonRemarks',
 'title',
 'type.1',
 'typeStatus',
 'type_x',
 'type_y',
 'verbatimDepth',
 'verbatimElevation',
 'verbatimEventDate',
 'verbatimLatitude',
 'verbatimLongitude',
 'vernacularName',
 'waterBody']


merged = merged[merged_cols]

merged = merged.convert_dtypes()

merged = merged.assign(digitizationDate = merged.digitizationDate.astype(pd.StringDtype()))


merged.describe(include='all')

In [ ]:
merged.head()

In [ ]:
merged.info(verbose=True)

In [ ]:
output_dir = "/media/data_cifs/projects/prj_fossils/data/raw_data/2022-yale_fossil/metadata-merged"
output_filename = "1_multimedia_merged_with_occurrences_metadata_on_id.tsv"

merged.to_csv(Path(output_dir, output_filename), sep="\t", na_rep="NaN")

In [ ]:
loaded = pd.read_csv(Path(output_dir, output_filename), sep="\t", index_col=0) #, na_rep="NaN")

loaded = loaded.convert_dtypes()
loaded = loaded.assign(digitizationDate = loaded.digitizationDate.astype(pd.StringDtype()))

assert loaded.equals(merged)
# loaded.info(verbose=True)

In [ ]:
merged_multimedia_w_occurrences_df = merged

In [ ]:
%%time

# merged = dp.merge(mk.DataPanel.from_pandas(multimedia_df), how="left", left_on="file_ids", right_on="identifier")
df = dp.view().to_pandas().drop(columns=["imgs", "height", "width", "aspect_ratio"])

merged = df.merge(merged_multimedia_w_occurrences_df, how="left", left_on="file_ids", right_on="identifier")
print(f"{df.shape=}, {dp.shape=}, {merged.shape=}, {merged_multimedia_w_occurrences_df.shape=}")

In [ ]:
output_dir = "/media/data_cifs/projects/prj_fossils/data/raw_data/2022-yale_fossil/metadata-merged"
output_filename = "2_image_file_ids_matched_with_multimedia_identifier_column.tsv"

merged.to_csv(Path(output_dir, output_filename), sep="\t", na_rep="NaN")

In [ ]:
merged.head()

In [ ]:
merged.describe(include='all')

In [ ]:
columns = ['paths',
 'file_ids',
 'identifier',
 'id',
 'imgs',
 'height',
 'width',
 'aspect_ratio',
 'type',
 'type.1',
 'title',
 'MetadataDate',
 'metadataLanguage',
 'providerManagedID',
 'comments',
 'rights',
 'rights.1',
 'WebStatement',
 'creator',
 'creator.1',
 'digitizationDate',
 'accessURI',
 'format',
 'format.1']

merged[columns]

In [ ]:
dp

In [ ]:
merged.digitizationDate.value_counts()#dtype

In [ ]:
loaded.digitizationDate.value_counts()#dtype

In [ ]:
loaded.describe(include='all')

In [ ]:
a = merged
b = loaded

comparison_result = (a == b) | ((a != a) & (b != b))

comparison_result

# a[a.digitizationDate != b.digitizationDate]


ab = a.merge(b, on="id")#, indicator
ab = ab[sorted(ab.columns)]

ab[ab.digitizationDate_x != ab.digitizationDate_y]
# b.digitizationDate

ab = ab.convert_dtypes()
# b.digitizationDate


ab.info(verbose=True)
# b.digitizationDate

comparison_result.all()

In [ ]:
list(merged.columns)

In [ ]:
occurrence_df[occurrence_df.id.apply(lambda x: x in merged.id)]

In [ ]:
%%time

# merged = dp.merge(mk.DataPanel.from_pandas(multimedia_df), how="left", left_on="file_ids", right_on="identifier")
df = dp.to_pandas()

merged = df.merge(multimedia_df, how="left", left_on="file_ids", right_on="identifier")
print(f"{dp.shape=}, {merged.shape=}")

merged.describe(include='all')

In [ ]:
columns = ['paths',
 'file_ids',
 'identifier',
 'id',
 'imgs',
 'height',
 'width',
 'aspect_ratio',
 'type',
 'type.1',
 'title',
 'MetadataDate',
 'metadataLanguage',
 'providerManagedID',
 'comments',
 'rights',
 'rights.1',
 'WebStatement',
 'creator',
 'creator.1',
 'digitizationDate',
 'accessURI',
 'format',
 'format.1']

merged[columns]

In [ ]:
# print(occurrence_df.shape)
# occurrence_df.describe(include='all')

### plots

In [ ]:
#!pip3 install streamlit-aggrid
# !pip3 install streamlit-pandas-profiling

In [ ]:
from streamlit_pandas_profiling import st_profile_report

In [ ]:
st_profile_report

In [ ]:
#Create two columns with different width
col1, col2 = st.columns( [0.8, 0.2])
with col1:               # To display the header text using css style
    st.markdown(""" <style> .font {
    font-size:35px ; font-family: 'Cooper Black'; color: #FF9633;} 
    </style> """, unsafe_allow_html=True)
    st.markdown('<p class="font">Upload your photo here...</p>', unsafe_allow_html=True)
    
with col2:               # To display brand logo
    st.image(image,  width=150)

In [ ]:
import missingno as msno
%matplotlib inline

In [ ]:
%%time

msno.matrix(occurrence_df, sort='descending') #.sample(2000))

In [ ]:
%%time
import matplotlib.pyplot as plt

plt.figure(figsize=(35,30))
msno.heatmap(occurrence_df, sort='descending', ax=plt.gca())#.sample(2000))

In [ ]:
%%time
import matplotlib.pyplot as plt

# plt.figure(figsize=(35,30))
msno.dendrogram(occurrence_df, orientation="top", figsize=(40,20))

In [ ]:
%%time

msno.bar(occurrence_df, sort='descending')#.sample(2000))

In [ ]:
dir(msno)

In [ ]:




dir(mk.DataPanel)

In [ ]:
mk.DataPanel
merged = dp.merge(mk.DataPanel.from_pandas(occurrence_df), how="left", left_on="file_ids", right_on="id")

print(f"{dp.shape=}, {merged.shape=}")

In [ ]:
%%time

merged_df = merged.to_pandas()

merged_df.describe(include='all')

In [ ]:
merged.to_pandas().describe(include='all')

In [ ]:
merged.columns

### Fuzzy matching

In [ ]:
#!pip3 install fuzzy_pandas
!pip3 install fuzzymatcher

In [ ]:
import pandas as pd
# import fuzzy_pandas as fpd

df1 = df.iloc[:10,:]
# df2 = occurrence_df

df2 = multimedia_df

import fuzzymatcher

In [ ]:
merged = fuzzymatcher.fuzzy_left_join(df1, df2,
                             left_on = "file_ids",
                             right_on = "identifier")
                             # right_on = "id")

merged

In [ ]:
merged.columns

In [ ]:
merged.sort_values("best_match_score", ascending=False)

In [ ]:
merged = fpd.fuzzy_merge(df1, df2,
                         left_on=['file_ids'],
                         right_on=['id'],
                         method="levenshtein",
                         ignore_case=True,
                         keep='match')
merged

In [ ]:
import pandas as pd
import fuzzy_pandas as fpd

df1 = df.iloc[:10,:]
df2 = occurrence_df

merged = fpd.fuzzy_merge(df1, df2,
                         left_on=['file_ids'],
                         right_on=['id'],
                         method="levenshtein",
                         ignore_case=True,
                         keep='match')
merged

In [ ]:
!head -n 1 {metadata_dir}/occurrence.txt

In [ ]:
multimedia_df = pd.read_csv(Path(metadata_dir, "multimedia.txt"), delimiter="\t", nrows=9449)
# occurrence_df = pd.read_json(Path(metadata_dir, "occurrence.txt"), skiprows)

multimedia_df.head()

In [ ]:
multimedia_dp = extract_dataset_from_file_list(fpaths = multimedia_df.accessURI)
multimedia_dp.lz



In [ ]:
occurrence_df = pd.read_csv(Path(metadata_dir, "occurrence.txt"), delimiter="\t", nrows=9449)
# occurrence_df = pd.read_json(Path(metadata_dir, "occurrence.txt"), skiprows)

occurrence_df.head()

In [ ]:
occurrence_df.describe(include='all')

In [ ]:
occurrence_df.columns.values

In [ ]:
# dir(Path(fpaths[k][0]))
# Path(fpaths[k][0]).stem
# Path(fpaths[k][0]).name









# Misc

In [ ]:
# from supervenn import supervenn
# import matplotlib.pyplot as plt
# import seaborn as sns

# style_list = ['default', 'classic'] + sorted(
#     style for style in plt.style.available if style != 'classic'
# )

# pp(style_list)
# style_label = "seaborn-notebook"
# plt.style.context(style_label)



# Plot a demonstration figure for every available style sheet.
# for style_label in style_list:
    # with plt.rc_context({"figure.max_open_warning": len(style_list)}):
        # with plt.style.context(style_label)





    
    
    
# for k,v in yale_fossil_dirs.items():
#     fpaths[k] = [os.path.join(v, p) for p in os.listdir(v)]
#     dps[k] = mk.DataPanel({
#         "paths" : fpaths[k],
#         "file_ids" : [Path(p).stem for p in fpaths[k]],
#         "imgs" : mk.ImageColumn.from_filepaths(fpaths[k])
#     })
#     dps[k]["height"] = dps[k]["imgs"].to_lambda(fn=lambda x: x.height)
#     # dps[k]["mean"] = dps[k]["imgs"].to_lambda(fn=lambda x: Stat(x).mean)
#     dps[k]["width"] = dps[k]["imgs"].to_lambda(fn=lambda x: x.width)